In [ ]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

# Read recipe inputs
df_RFM_histo_cleaned = dataiku.Dataset("df_RFM_histo_cleaned")
df = df_RFM_histo_cleaned.get_dataframe()
df["periode"]=pd.to_datetime(df["periode"])
df["periode"]=df["periode"].dt.to_period('M').astype(str)

# Compute recipe outputs from inputs
# TODO: Replace this part by your actual code that computes the output, as a Pandas dataframe
# NB: DSS also supports other kinds of APIs for reading and writing data. Please see doc.
df.drop(columns="recency",inplace=True)
df = df.sort_values(by=['client_id','periode'])
df["next_segment"] = df.groupby("client_id")["segment"].shift(-1)
df["next_period"] = df.groupby("client_id")["periode"].shift(-1)
df_transitions = df.dropna(subset=['next_segment'])
sankey_data = (
df_transitions
    .groupby(["periode","next_period","segment","next_segment"])
    .agg(Nb_clients=('client_id','nunique'))
    .reset_index()
)
# Write recipe outputs
df_prep_sankey = dataiku.Dataset("df_prep_sankey")
df_prep_sankey.write_with_schema(sankey_data)
